In [1]:
import os
from dotenv import load_dotenv
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Qdrant
from langchain.document_loaders import TextLoader

load_dotenv()

True

In [2]:
from pathlib import Path

files = list(Path("docs").glob("**/*.pdf"))
files

[PosixPath('docs/forms/form10-k.pdf'),
 PosixPath('docs/forms/form8-k.pdf'),
 PosixPath('docs/law/sec-adopts-mandatory-cybersecurity-disclosure-rules.pdf'),
 PosixPath('docs/law/33-11216.pdf'),
 PosixPath('docs/law/33-11216-fact-sheet.pdf'),
 PosixPath('docs/soc2_reports/pwm-auditors-report-gs007-controls-assurance-report_2019 (1).pdf'),
 PosixPath('docs/soc2_reports/Microsoft_General_-_NGP_PIMS_SSAE_18_SOC_2_Report.pdf'),
 PosixPath('docs/accounting_audit/408147977-Sample-Report-PwC.pdf'),
 PosixPath('docs/policies/Cybersecurity-committee-Charter.pdf'),
 PosixPath('docs/policies/CYBERSECURITY RISK MANAGEMENT COMMITTEE CHARTER 2017.pdf'),
 PosixPath('docs/policies/cisa_csac_renewal_charter_cmomm_filed_23may2023_508_0.pdf'),
 PosixPath('docs/policies/Hierarchical-Cybersecurity-Governance-Framework.pdf'),
 PosixPath('docs/policies/CG_docs.pdf'),
 PosixPath('docs/policies/CYBERSECURITY-SUBCOMMITTEE-CHARTER.pdf'),
 PosixPath('docs/policies/Cybersecurity-Committee-Charter (1).pdf'),
 PosixP

In [ ]:
from services.nougart import query_hugginface_inference_api

In [4]:
from services.convert import pdf_to_markdown
from pprint import pprint

text = pdf_to_markdown(files[0])

(' UNITED STATES\n'
 ' SECURITIES AND EXCHANGE COMMISSION\n'
 ' Washington, D.C. 20549\n'
 ' FORM 10-K\n'
 ' ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF\n'
 ' THE SECURITIES EXCHANGE ACT OF 1934\n'
 ' GENERAL INSTRUCTIONS\n'
 'A. Rule as to Use of Form 10-K.\n'
 '(1)  This Form shall be used for annual reports pursuant to Section 13 or '
 '15(d) of the Securities Exchange Act of 1934 (15 \n'
 'U.S.C. 78m or 78o(d)) (the “Act”) for which no other form is prescribed. '
 'This Form also shall be used for transition \n'
 'reports ﬁ led pursuant to Section 13 or 15(d) of the Act. \n'
 '(2)  Annual reports on this Form shall be ﬁ led within the following '
 'period: \n'
 '(a) 60 days after the end of the ﬁ scal year covered by the report (75 days '
 'for ﬁ scal years ending before December \n'
 '15, 2006) for large accelerated ﬁ lers (as deﬁ ned in 17 CFR 240.12b-2): \n'
 ' \n'
 '(b)   75 days after the end of the ﬁ scal year covered by the report for '
 'accelerated ﬁ lers (as deﬁ ned 

In [ ]:
qdrant = Qdrant.from_documents(
    docs,
    embeddings,
    url=url,
    prefer_grpc=True,
    api_key=os.getenv("QDRANT_API_KEY"),
    collection_name="my_documents",
)

In [3]:
from services.langchain.chains import load_chain

chain = load_chain()
with open("markdown/sample.md") as f:
    SAMPLE_DOCUMENT = f.read()

output = chain.run(SAMPLE_DOCUMENT)
output



> Entering new LLMChain chain...
Prompt after formatting:
System: 
Using the following examples please provide a criticism of my current cybersecurity policy :
```
RESPONSIBILITIES
The following are the principal recurring responsibilities of the Cybersecurity Committee. 
1. Information Technology and Network Systems. The Cybersecurity Committee shall oversee
the quality and effectiveness of the Company’s policies and procedures with respect to its
information technology and network systems, including encryption, network security and data
security, as well as access to such systems.
2. IT/Engineering Security Funding. The Cybersecurity Committee shall oversee the Company’s
information technology senior management team relating to budgetary priorities based, in part, on
assessing risk associated with various perceived threats.
3. Incident Response. The Cybersecurity Committee shall review and provide oversight on the
policies and procedures of the Company in preparation for responding

'The cybersecurity policy presented above is comprehensive but there are a few areas that could be improved upon:\n\n1. **Lack of specifics in Incident Response**: The policy does not provide specific guidelines on the incident response process. It is important to define what constitutes an incident, who is responsible for what during an incident, and the steps to be taken after an incident.\n\n2. **No Mention of Regular Policy Updates**: Cybersecurity landscapes are constantly evolving with new threats emerging on a regular basis. There is no mention of regular updates or revisions of the policy to keep it up-to-date with current threats and best practices.\n\n3. **No Emphasis on Employee Training**: While the policy does mention cybersecurity employee training, it does not emphasize the importance of regular and comprehensive cybersecurity training for all employees, not just IT staff, which is crucial for a robust cybersecurity posture.\n\n4. **Absence of Clear Communication Channel

In [1]:
from langchain.output_parsers import PydanticOutputParser
from langchain.pydantic_v1 import BaseModel, Field
from typing import List
from langchain.chains.openai_functions.utils import get_llm_kwargs



class Critisim(BaseModel):
    """
    A critisim of a security policy document.
    """

    critisim: str = Field(..., description="The critisim of the policy document in reference to context from the document. This should be in answer to the system prompt")
    context: List[str] = Field(
        ...,
        description=(
            "The context for the critisim."
        ),
    )

output_parser = PydanticOutputParser(pydantic_schema=Critisim)
schema = Critisim.schema()
function = {
    "name": schema["title"],
    "description": schema["description"],
    "parameters": schema,
}
llm_kwargs = get_llm_kwargs(function)
output_parser.parse("output")

ValidationError: 1 validation error for PydanticOutputParser
pydantic_object
  field required (type=value_error.missing)